In [1]:
import numpy as np
import pandas as pd
import os

from pycausal.pycausal import pycausal as pc

import random
import re
import copy

from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import cross_val_score



In [2]:

filename = "credit_clean.csv"

sample_number = 3000
protected_attribute = 'gender'

In [3]:
def find_range_cols(data):
    range_col = [] # columns that fit numerical values into ranges
    for col in data.select_dtypes(exclude=['int64', 'float64']).columns:
        if any(item in data[col][0] for item in ['>', '<']): 
            range_col.append(col)
                
    return range_col

class DataEncoder(object):
    def __init__(self, class_column='class', cat_columns=None):
        self.class_column = class_column
        self.cat_columns = cat_columns

        # these will be trained with fit_encoders()
        self.column_encoders = {} # label encoder
        self.cat_encoder = None # one-hot encoder
        self.label_encoder = None # label encoder

    def fit(self, data):
        """
        Fit one-hot encoders for categorical features and an integer encoder for
        the label. These can be used later to transform raw data into a form
        that ATM can work with.

        data: pd.DataFrame of unprocessed data
        """
        if self.class_column not in data.columns:
            raise KeyError('Class column "%s" not found in dataset!' %
                           self.class_column)
            
        range_col = find_range_cols(data)
                
        self.range_col = range_col
        print(range_col)
            

        # encode categorical columns, leave ordinal values alone
        if self.cat_columns is None:
            cats = data.drop([self.class_column]+range_col, axis=1).select_dtypes(exclude=['int64'])
            self.cat_columns = cats.columns
        else:
            cats = data[self.cat_columns].drop(range_col, axis=1).select_dtypes(exclude=['int64'])
            
        self.cat_cols = cats.columns
#         print(cats.columns)
        
        for cat_name in cats.columns:   
        # save the indices of categorical columns for one-hot encoding

            # encode each feature as an integer in range(unique_vals)
            le = LabelEncoder()
            cats[cat_name] = le.fit_transform(cats[cat_name])
            self.column_encoders[cat_name] = le

        # One-hot encode the whole feature matrix.
        # Set sparse to False so that we can test for NaNs in the output
        self.cat_encoder = OneHotEncoder(sparse=False)
        # if Category column exists          
        if cats.shape[1] != 0:
            self.cat_encoder.fit(cats)

        # Train an encoder for the label as well
        labels = np.array(data[[self.class_column]])
        self.label_encoder = LabelEncoder()
        self.label_encoder.fit(labels)
        

    def transform(self, data):
        """
        Convert a DataFrame of labeled data to a feature matrix in the form
        that ATM can use.
        """
        y = self.transform_y(data)
        X = self.transform_x(data)

        return X, y
    
    def transform_x(self, data):
        """
        only transform x, for the generated data
        """
        cats = data[self.cat_columns]

        # encode each categorical feature as an integer
        for column, encoder in list(self.column_encoders.items()):
            cats[column] = encoder.transform(cats[column])

        # one-hot encode the categorical features
        if cats.shape[1] != 0:
            X = self.cat_encoder.transform(cats)
        else:
            X = cats
        
#         print(X)
        if self.class_column in data:
            nums = data.drop([self.class_column], axis=1).select_dtypes(include=['int64']).values
        else:
            nums = data.select_dtypes(include=['int64']).values
            
       
        # transform range cols into integrate. e.g., <4 -> 1; 4<x<7 -> 2
        ranges = []
        for col in self.range_col:
            values = data[col]
            ranges.append( self.range2int(values) )
        ranges = np.transpose( np.array(ranges) )
#         print(X.shape, nums.shape, ranges.shape)
        
        X = np.concatenate((X, nums, ranges), axis=1)
        return X
    
    def transform_y(self, data):
        if self.class_column in data:
            # pull labels into a separate series and transform them to integers
            labels = np.array(data[[self.class_column]])
            y = self.label_encoder.transform(labels)
            # drop the label column and transform the remaining features
        else:
            y = None
            
        return y
    
    def range2int(self, values):
        ranges = []
        for v in values:
            if v not in ranges:
                ranges.append(v)
        
        def sort_key(x):
            num_strings = re.findall('\d+', x)
            # 'undefined' is in the front
            if len(num_strings)==0:
                return -1
            # x> 1, x<7
            elif len(num_strings)==1:
                return int(num_strings[0])*2
            # 1<x<7
            else:
                nums = map(int, num_strings) # string to number
                return sum(nums)
                
        ranges.sort(key=sort_key)
        return list(map(lambda x: ranges.index(x), values))
        
        
    
    def fit_transform(self, data):
        """ Process data into a form that ATM can use. """
        self.fit(data)
        return self.transform(data)

In [4]:
data = pd.read_csv(filename)

# drop rows with any NA values
data = data.dropna(how='any')
data = data[0:1000]

encoder = DataEncoder()
encoder.fit(data)
x_train, y_train = encoder.transform(data)



['checking_status', 'duration', 'credit_amount', 'employment', 'age']


c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\administrator\appdata\local\programs\python\python37\lib\site-pac

In [5]:
svm_cls = SVC(
    C = 0.0484055070919,
    cache_size = 15000,
    class_weight = "balanced",
    gamma = 424.665365448,
    kernel = "rbf",
    max_iter = 50000,
    probability = True,
    shrinking = True)


mlp_cls = MLPClassifier(
    activation = "relu",
    alpha = 0.00146693288878,
    batch_size = "auto",
    hidden_layer_sizes= (189,50, ),
    learning_rate = "constant",
    learning_rate_init = 0.898892816545,
    solver = "sgd"
)

rf_cls = RandomForestClassifier(
    bootstrap=True, 
    class_weight=None, 
    criterion='entropy', 
    max_depth=10, 
    max_features=0.45, 
    max_leaf_nodes=None, 
    min_impurity_decrease=1e-07, 
    min_samples_leaf=6, 
    min_samples_split=7, 
    min_weight_fraction_leaf=0.0, 
    n_estimators=512, 
    n_jobs=1, 
    oob_score=False, 
    random_state=3, 
    verbose=0, 
    warm_start=False
)

knn_cls = KNeighborsClassifier(
    algorithm = "ball_tree",
    leaf_size = 40,
    metric = "manhattan",
    n_neighbors = 17
)



In [6]:
# svm_score = cross_val_score(svm_cls, x_train, y_train, scoring='accuracy', cv=10) 
# mlp_score = cross_val_score(mlp_cls, x_train, y_train, scoring='accuracy', cv=10) 

knn_cls.fit(x_train, y_train)
knn_score = cross_val_score(knn_cls, x_train, y_train, scoring='accuracy', cv=10) 

rf_cls.fit(x_train, y_train)
rf_score = cross_val_score(rf_cls, x_train, y_train, scoring='accuracy', cv=10) 

print('knn', knn_score, 'rf', rf_score)


knn [0.74 0.71 0.72 0.76 0.76 0.74 0.67 0.74 0.75 0.71] rf [0.83 0.71 0.72 0.77 0.77 0.8  0.71 0.81 0.75 0.77]


In [7]:
# Data generator
class DataGene(object):
    
    def __init__(self, data, sample_num=10, class_col='class'):
        self.data = data
        self.sample_num = sample_num
        self.class_col = 'class'
        
    def get_samples(self):
        features = self.data.drop([self.class_col], axis=1)
#         print(features)
        range_cols = find_range_cols(self.data)
        cat_cols = features.select_dtypes(exclude=['int64']).drop(range_cols, axis=1).columns
        num_cols = features.select_dtypes(include=['int64']).columns
        print(cat_cols)
        print(num_cols)
        samples = []
        
        for i in range(self.sample_num):
            sample_cat = [
                random.choice(encoder.column_encoders[cat_name].classes_)
                for cat_name in cat_cols
            ]
            sample_num = [
                random.choice( list(set(features[num_name])) )
                for num_name in num_cols
            ]
            sample_range = [
                random.choice( list(set(features[range_name])) )
                for range_name in range_cols
            ]
            sample = sample_cat + sample_num + sample_range
            samples.append(sample)
            
        
        samples = pd.DataFrame(samples, columns=list(cat_cols)+list(num_cols)+range_cols)
        return samples        
    

In [8]:
# generate new samples
dataGene = DataGene(data, sample_num=sample_number)
new_samples = dataGene.get_samples()
# encode sample data
x_samples, y_samples = encoder.transform(new_samples)


Index(['marriage', 'gender', 'credit_history', 'purpose', 'savings_status',
       'other_parties', 'property_magnitude', 'other_payment_plans', 'housing',
       'job', 'own_telephone', 'foreign_worker'],
      dtype='object')
Index(['installment_commitment', 'residence_since', 'existing_credits',
       'num_dependents'],
      dtype='object')


c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [9]:
# generate samples to describe model behavior
def add_col(df, new_col, col_name):
    new_df = df.copy()
    new_df[col_name] = pd.Series(np.asarray(new_col), index= df.index) 
    return new_df

y_samples_knn = knn_cls.predict(x_samples)
y_samples_rf = rf_cls.predict(x_samples)


# pandas data frame
knn_samples = add_col(new_samples, y_samples_knn, 'class') 
rf_samples = add_col(new_samples, y_samples_rf, 'class')

model_samples = knn_samples





In [10]:
# find key attributes
import re
from pycausal.pycausal import pycausal as pc
from pycausal import search as s

pc = pc()
pc.start_vm()



def findKeyAttrs(samples, protect_attr, result_attr = 'class'):
    """
    Args:
        samples(pandas data frames): 
        protect_attr(string || Array<string>): 
    Return:
        key_attrs(list<string>): a list of key attributes that directly influence the decision
    """
    key_attrs = []
    ### use bayes Est to find the key attributes
    ### slow, extract more key attributes 
#     graph = s.bayesEst(samples, depth = 0, alpha = 0.05, verbose = True)
    ### OR use Fast Greedy Equivalence Search
    ### faster than bayes, get less key attributes
    graph = s.tetradrunner()
    graph.getAlgorithmParameters(algoId = 'fges', scoreId = 'bdeu')

    graph.run(algoId = 'fges', dfs = data, scoreId = 'bdeu', priorKnowledge = None, dataType = 'discrete',
           structurePrior = 0.5, samplePrior = 0.5, maxDegree = 5, faithfulnessAssumed = True, verbose = False)
    
#     graph.getNodes()
    
    for edge in graph.getEdges():
        if 'class' in edge:
            # extract attr name from the edge
            # remove --> or --o or --- and white space
            attr = re.sub(r'-+>?o?|{}|\s+'.format(result_attr), '', edge)
            key_attrs.append(attr)
            
            
    # remove protect attrs        
    if type(protect_attr) is not str: 
        # if protect attr is a list
        for a in protect_attr:
            if a in key_attrs:
                key_attrs.remove(a)
    elif protect_attr in key_attrs:
        # if protect attr is a string
        key_attrs.remove(protect_attr)
    return key_attrs


In [11]:
# candidate attribute
candi_attrs = ['housing']
candi_vals = {}
for candi_attr in candi_attrs:
    candi_vals[candi_attr] = list(set(model_samples[candi_attr]))

# identify candidate groups
protect_attr = protected_attribute
protect_vals = list(set(model_samples[protect_attr]))

key_attrs = findKeyAttrs(model_samples, protect_attr)
key_vals = {}
for key_attr in key_attrs:
    key_vals[key_attr] = list(set(model_samples[key_attr]))
    
all_groups = []

all_attrs = key_attrs + candi_attrs
all_vals = {}
for all_attr in all_attrs:
    all_vals[all_attr] = list(set(model_samples[all_attr]))

print('\n')
print('protected attribute:', protect_attr, protect_vals, '\n')
print('key attributes:', key_attrs, '\n')
print(key_vals, '\n')
print('candidate attribute:', candi_attrs, candi_vals, '\n')


def generate_groups(all_vals, depth, index, all_groups):
    '''
    Args: 
        key_vals(dict): e.g., {'atrribute_a':['a', 'b', 'c'], 'attribute_b': ['m', 'n', 'k']}
        depth(int): recursive depth
        index(dict): the choosen value of each key attribute. e.g.,  {'atrribute_a':'a', 'attribute_b': 'k'}
        all_groups: a list of index
    
    '''
#     print('a', key_vals, key_attrs, depth)
    for k in all_vals[ all_attrs[depth] ]:
        index_ = copy.deepcopy(index)
        index_[all_attrs[depth]] = k
        if depth < len(all_attrs)-1: 
            depth_ = depth+1 
            generate_groups(all_vals, depth_, index_, all_groups)
        else:
            all_groups.append(index_)
    

generate_groups(all_vals, 0, {}, all_groups)
# print(key_groups)

for group in all_groups:
    group_items = model_samples.copy()
    for attr in group:
        group_items = group_items.loc[group_items[attr]==group[attr]]
    print(group, len(group_items))
    if len(group_items)>0:
        for val in protect_vals:
            # based on protected attribute
            group_items_ = group_items.loc[group_items[protect_attr] == val]
            if len(group_items)>0:
#                 group_reject = group_items_.loc[group_items_['class'] == 0]
#                 group_accept = group_items_.loc[group_items_['class'] == 1]
#                 p_0 = len(group_reject)/len(group_items_)
#                 p_1 = len(group_accept)/len(group_items_)
#                 print(val, "{:.2f}".format(p_0), "{:.2f}".format(p_1), len(group_items_))
                group_high= group_items_.loc[group_items_['class'] == 0]
                group_low = group_items_.loc[group_items_['class'] == 1]
#                 group_middle = group_items_.loc[group_items_['class'] == 2]
                p_1 = len(group_low)/len(group_items_)
#                 p_2 = len(group_middle)/len(group_items_)
                p_0 = len(group_high)/len(group_items_)
#                 print(val, "{:.2f}".format(p_1), "{:.2f}".format(p_2), "{:.2f}".format(p_0), len(group_items_))
                print(val, "{:.2f}".format(p_1), "{:.2f}".format(p_0), len(group_items_))

samplePrior: Sample prior (min = 1.0) (java.lang.Double) [default:1.0]
structurePrior: Structure prior coefficient (min = 1.0) (java.lang.Double) [default:1.0]
faithfulnessAssumed: Yes if (one edge) faithfulness should be assumed (java.lang.Boolean) [default:true]
symmetricFirstStep: Yes if the first step step for FGES should do scoring for both X->Y and Y->X (java.lang.Boolean) [default:false]
maxDegree: The maximum degree of the graph (min = -1) (java.lang.Integer) [default:100]
verbose: Yes if verbose output should be printed or logged (java.lang.Boolean) [default:true]
numberResampling: The number of resampling iterations (min = 0) (java.lang.Integer) [default:0]
resampleSize: The resample size (min = 1) (java.lang.Integer) [default:1]
resamplingWithReplacement: Yes, if resampling with replacement (bootstrapping) (java.lang.Boolean) [default:true]
resamplingEnsemble: Ensemble method: Preserved (0), Highest (1), Majority (2) (java.lang.Integer) [default:1]


protected attribute: gen

{'checking_status': 'no checking', 'credit_history': 'critical/other existing credit', 'housing': 'own'} 53
female 0.96 0.04 23
male 1.00 0.00 30
{'checking_status': 'no checking', 'credit_history': 'critical/other existing credit', 'housing': 'for free'} 48
female 0.96 0.04 26
male 0.95 0.05 22
{'checking_status': 'no checking', 'credit_history': 'critical/other existing credit', 'housing': 'rent'} 56
female 0.97 0.03 35
male 1.00 0.00 21
